In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# Import MNIST data from tensflow. Already downloaded and stored in MNIST_data folder
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist.train.num_examples

55000

In [5]:
mnist.train.images[1].shape

(784,)

In [6]:
tf.reset_default_graph()

# Graph for DNN (Just One layer with as many neurons as pixels in each flatten image)

In [7]:
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")
Y_OHE_true = tf.placeholder(tf.float32, [None,10], name = "Y_OHE_true")

In [8]:
W = tf.Variable(np.random.rand(784,10), dtype=tf.float32, name="W")
B = tf.Variable(np.random.rand(10), dtype=tf.float32, name="B")

In [9]:
# DAG
Z = tf.add(tf.matmul(X,W),B, name="Z")   # 10 outputs, one for each digit

In [10]:
#loss
losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z, labels=Y_OHE_true, name="losses")  # Array of losses Per batch

In [11]:
#Cost
cost = tf.reduce_sum(losses, name="Cost")     # Total loss per batch

In [12]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
set_to_optimize = optimizer.minimize(cost,name="To_Be_Optimized")

In [13]:
init = tf.global_variables_initializer()

In [14]:
writer = tf.summary.FileWriter(logdir="./graph", graph=tf.get_default_graph())

In [15]:
summ = tf.summary.scalar("Total_Cost", cost)   # Attach summary to node: cost

In [16]:
sess = tf.Session()
sess.run(init)

In [17]:
for i in range(1000):
    batch_x , batch_y = mnist.train.next_batch(100)   # Leveraging inbuilt mini-batch generator 
    sess.run(set_to_optimize, feed_dict={X:batch_x, Y_OHE_true:batch_y})
    s_buffer = sess.run(summ, feed_dict={X:batch_x, Y_OHE_true:batch_y})    # Cost of every mini-batch
    writer.add_summary(s_buffer, i)

In [18]:
predictions = sess.run(tf.argmax(Z, axis=1), feed_dict={X:mnist.test.images})

In [19]:
acc = (predictions == np.argmax(mnist.test.labels,axis=1))
acc

array([ True,  True,  True, ...,  True, False,  True])

In [20]:
acc.mean()

0.8869

In [21]:
sess.close()
writer.close()